In [3]:
import pandas as pd
import numpy as np
import sqlite3
import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader
import matplotlib.pyplot as plt
import seaborn as sns
import datetime
import pickle
import os

# Set a professional plotting style
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_context("paper", font_scale=1.5)

# Physical constants
k_b = 1.380649e-23  # Boltzmann constant (J/K)
q = 1.60217662e-19  # Elementary charge (C)
T_STC = 25.0  # Standard Test Conditions temperature (°C)
G_STC = 1000.0  # Standard Test Conditions irradiance (W/m²)
T_ref_K = T_STC + 273.15  # Reference temperature in Kelvin

# LNN Model Definition (included for consistency, though not used for recalculation)
def get_activation(name):
    if name == "tanh":
        return torch.tanh
    elif name == "relu":
        return torch.nn.functional.relu
    elif name == "leaky_relu":
        return lambda x: torch.nn.functional.leaky_relu(x, negative_slope=0.01)
    elif name == "gelu":
        return torch.nn.functional.gelu
    else:
        raise ValueError(f"Unsupported activation: {name}")

class LiquidTimeStep(nn.Module):
    def __init__(self, input_size, hidden_size, activation_name):
        super().__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.W_in = nn.Linear(input_size, hidden_size)
        self.W_h = nn.Linear(hidden_size, hidden_size)
        self.tau = nn.Parameter(torch.ones(hidden_size) * 0.1)
        self.activation = get_activation(activation_name)

    def forward(self, x, h, dt=0.1):
        dx = self.activation(self.W_in(x) + self.W_h(h))
        h_new = h + dt * (dx - h) / self.tau
        return h_new

class LiquidNeuralNetwork(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, num_layers, activation_name, dropout_rate):
        super().__init__()
        self.hidden_size = hidden_size
        self.layers = nn.ModuleList([
            LiquidTimeStep(input_size if i == 0 else hidden_size, hidden_size, activation_name)
            for i in range(num_layers)
        ])
        self.bn = nn.BatchNorm1d(hidden_size)
        self.dropout = nn.Dropout(dropout_rate)
        self.output_layer = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        batch_size, seq_len, _ = x.size()
        h = torch.zeros(batch_size, self.hidden_size, device=x.device)
        for t in range(seq_len):
            h_t = x[:, t, :]
            for layer in self.layers:
                h = layer(h_t, h)
                h_t = h
        h = self.bn(h)
        h = self.dropout(h)
        return self.output_layer(h)

# Function to calculate zenith angle
def calculate_zenith_angle(timestamp, latitude, longitude, standard_meridian=135):
    lat_rad = np.radians(latitude)
    day_of_year = timestamp.timetuple().tm_yday
    declination = 23.45 * np.sin(np.radians(360 * (284 + day_of_year) / 365))
    decl_rad = np.radians(declination)
    B = (360 / 365) * (day_of_year - 81)
    EOT = 9.87 * np.sin(np.radians(2 * B)) - 7.53 * np.cos(np.radians(B)) - 1.5 * np.sin(np.radians(B))
    hour = timestamp.hour + timestamp.minute / 60.0
    time_correction = (4 * (longitude - standard_meridian) + EOT) / 60.0
    solar_time = hour + time_correction
    hour_angle = 15 * (solar_time - 12)
    hour_rad = np.radians(hour_angle)
    cos_zenith = (np.sin(lat_rad) * np.sin(decl_rad) +
                  np.cos(lat_rad) * np.cos(decl_rad) * np.cos(hour_rad))
    zenith_angle = np.degrees(np.arccos(np.clip(cos_zenith, -1, 1)))
    return zenith_angle

# Define PV module parameters (example Monocrystalline)
PV_MODULES = {
    "Monocrystalline": {
        "Isc_ref": 9.0,  # Short-circuit current at STC (A)
        "Voc_ref": 37.0,  # Open-circuit voltage at STC (V)
        "Imp_ref": 8.5,   # Maximum power point current at STC (A)
        "Vmp_ref": 30.0,  # Maximum power point voltage at STC (V)
        "Pmp_ref": 255.0, # Maximum power at STC (W)
        "alpha_Isc": 0.0005,  # Temperature coefficient of Isc (1/°C)
        "beta_Voc": -0.12,    # Temperature coefficient of Voc (V/°C)
        "a_ref": 1.3,         # Diode ideality factor
        "I0_ref": 1e-9,       # Diode reverse saturation current at STC (A)
        "Rs_ref": 0.3,        # Series resistance at STC (Ω)
        "Rsh_ref": 500,       # Shunt resistance at STC (Ω)
        "gamma_Rs": -0.0001,  # Temperature coefficient of Rs (1/°C)
        "cells": 60,          # Number of cells in series
        "area_ref": 1.6,      # Reference area (m²)
        "u0": 26.91,          # NOCT coefficient (W/m²°C)
        "u1": 6.2             # NOCT coefficient (W/m²°C)
    }
}

# Define actual paths based on your offline modeling code
model_save_dir = r"D:\Masters thesis PV\Python codes\trained_models_spatiotemporal_ghi"
save_dir = r"D:\Masters thesis PV\Final input dataset for LNN\preprocessed_spatiotemporal_ghi"
model_path = os.path.join(model_save_dir, 'lnn_3hour_spatiotemporal_ghi.pth')
scaler_X_path = os.path.join(save_dir, 'scaler_X_spatiotemporal_ghi.pkl')
scaler_y_path = os.path.join(save_dir, 'scaler_y_spatiotemporal_ghi.pkl')
best_params_path = os.path.join(model_save_dir, 'best_params_spatiotemporal_ghi.pkl')
data_info_path = os.path.join(save_dir, 'data_info_spatiotemporal_ghi.txt')
db_path = os.path.join(r"D:\Masters thesis PV\Data collection\NMSC Web Page\ghi_data_2023_2024.db")

# Load LNN model and scalers (for consistency, though not used for recalculation)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
prefix = 'spatiotemporal_ghi'

with open(best_params_path, "rb") as f:
    best_params = pickle.load(f)

with open(data_info_path, "r") as f:
    data_info = f.readlines()
    timesteps = int(data_info[0].split(": ")[1])
    n_features = int(data_info[1].split(": ")[1])

lnn_model = LiquidNeuralNetwork(
    input_size=n_features,
    hidden_size=best_params["hidden_size"],
    output_size=1,  # Adjusted to match checkpoint (single-step)
    num_layers=best_params["num_layers"],
    activation_name=best_params["activation"],
    dropout_rate=best_params["dropout_rate"]
).to(device)
lnn_model.load_state_dict(torch.load(model_path, map_location=device))
lnn_model.eval()

with open(scaler_X_path, "rb") as f:
    scaler_X = pickle.load(f)
with open(scaler_y_path, "rb") as f:
    scaler_y = pickle.load(f)

# Step 1: Load Preprocessed Test Dataset
conn = sqlite3.connect(db_path)
query_test = """
    SELECT *
    FROM test_data_spatiotemporal_advanced_with_time
    ORDER BY lat, lon, timestamp
"""
df_test = pd.read_sql_query(query_test, conn)
conn.close()

df_test['timestamp'] = pd.to_datetime(df_test['timestamp'])

# Debugging: Check df_test
print("df_test shape:", df_test.shape)
print("df_test columns:", df_test.columns.tolist())

# Step 2: Select a Location with Sufficient Data
desired_lat, desired_lon = 37.5, 126.9
lat_tolerance = 0.0001
lon_tolerance = 0.0001
df_loc = df_test[
    (df_test['lat'].between(desired_lat - lat_tolerance, desired_lat + lat_tolerance)) &
    (df_test['lon'].between(desired_lon - lon_tolerance, desired_lon + lon_tolerance))
].copy()

if len(df_loc) < 1:
    print("Initial selected location has insufficient data. Finding a location with enough data...")
    location_counts = df_test.groupby(['lat', 'lon']).size().reset_index(name='count')
    viable_locations = location_counts[location_counts['count'] >= 1]
    if viable_locations.empty:
        raise ValueError("No locations in the dataset have enough data.")
    viable_locations['lat_diff'] = np.abs(viable_locations['lat'] - desired_lat)
    viable_locations['lon_diff'] = np.abs(viable_locations['lon'] - desired_lon)
    viable_locations['total_diff'] = viable_locations['lat_diff'] + viable_locations['lon_diff']
    best_location = viable_locations.loc[viable_locations['total_diff'].idxmin()]
    lat = best_location['lat']
    lon = best_location['lon']
    print(f"Selected new location with sufficient data - lat: {lat}, lon: {lon}, rows: {best_location['count']}")
    df_loc = df_test[
        (df_test['lat'].between(lat - lat_tolerance, lat + lat_tolerance)) &
        (df_test['lon'].between(lon - lon_tolerance, lon + lon_tolerance))
    ].copy()

# Step 3: Recalculate IEC Metrics
module = PV_MODULES["Monocrystalline"]
time_interval = 0.5  # 30 minutes

# Identify GHI columns
ghi_current_column = next((col for col in df_loc.columns if col.startswith('GHI') and '(t)' in col), None)
ghi_target_column = 'GHI_t_plus_6_timesteps'  # Target column from training

if not ghi_current_column:
    raise KeyError(f"No GHI column with (t) suffix found. Available columns: {df_loc.columns.tolist()}")
print(f"Using {ghi_current_column} as current GHI column")

if ghi_target_column not in df_loc.columns:
    print(f"Warning: {ghi_target_column} not found. Using {ghi_current_column} as fallback for actual GHI")
    ghi_target_column = ghi_current_column

# Extract GHI data
ghi_current = df_loc[ghi_current_column].values
ghi_target = df_loc[ghi_target_column].values

# Identify zenith and azimuth columns with flexible detection
zenith_columns = [col for col in df_loc.columns if col.startswith('Zenith_Angle')]
azimuth_columns = [col for col in df_loc.columns if col.startswith('Solar_Azimuth')]

if not zenith_columns or not azimuth_columns:
    raise KeyError(f"Zenith or Azimuth column not found. Available columns: {df_loc.columns.tolist()}")
zenith_current_column = zenith_columns[0]  # Use the first matching column
azimuth_current_column = azimuth_columns[0]  # Use the first matching column
print(f"Using {zenith_current_column} for zenith and {azimuth_current_column} for azimuth")

zenith_current = df_loc[zenith_current_column].values
azimuth_current = df_loc[azimuth_current_column].values

# Calculate GTI
tilt = 37
orientation = 0
def compute_gti(ghi, dni, dhi, zenith, solar_azimuth, tilt, orientation, albedo=0.2):
    zenith_rad = np.radians(zenith)
    solar_azimuth = np.radians(solar_azimuth)
    tilt_rad = np.radians(tilt)
    orientation = np.radians(orientation)
    cos_theta_i = (np.sin(zenith_rad) * np.cos(tilt_rad) * np.cos(solar_azimuth - orientation) +
                   np.cos(zenith_rad) * np.sin(tilt_rad))
    cos_theta_i = max(0, cos_theta_i)
    # Simplified assumption if DNI/DHI are not available
    dni = ghi * 0.8  # Placeholder
    dhi = ghi * 0.2  # Placeholder
    beam_tilted = dni * cos_theta_i
    diffuse_tilted = dhi * (1 + np.cos(tilt_rad)) / 2
    reflected = albedo * ghi * (1 - np.cos(tilt_rad)) / 2
    return max(0, beam_tilted + diffuse_tilted + reflected)

gti_current = np.array([compute_gti(ghi, 0, 0, zenith, azimuth, tilt, orientation) 
                       for ghi, zenith, azimuth in zip(ghi_current, zenith_current, azimuth_current)])
gti_target = np.array([compute_gti(ghi, 0, 0, zenith, azimuth, tilt, orientation) 
                      for ghi, zenith, azimuth in zip(ghi_target, zenith_current, azimuth_current)])

# Calculate cell temperature
temperature_column = next((col for col in df_loc.columns if col.startswith('temperature') and '(t)' in col), None)
if not temperature_column:
    raise KeyError(f"Temperature column not found. Available columns: {df_loc.columns.tolist()}")
temperature_current = df_loc[temperature_column].values
u0, u1 = module["u0"], module["u1"]
cell_temperature = temperature_current + (gti_current / (u0 + u1))

# Compute PV power
def compute_pv_power(gti, temp_cell, pv_area, module):
    T_cell_kelvin = temp_cell + 273.15
    IL = module["Isc_ref"] * (gti / G_STC) * (1 + module["alpha_Isc"] * (temp_cell - T_STC))
    I0 = module["I0_ref"] * (T_cell_kelvin / T_ref_K)**3 * np.exp((q * 1.12 / k_b) * (1 / T_ref_K - 1 / T_cell_kelvin))
    I0 = np.clip(I0, 1e-12, 1e-6)
    a = module["a_ref"] * (T_cell_kelvin / T_ref_K)
    Rs = module["Rs_ref"] * (1 + module["gamma_Rs"] * (temp_cell - T_STC))
    Rsh = module["Rsh_ref"] * (G_STC / max(gti, 1e-6))
    Rsh = np.clip(Rsh, 100, 1e6)
    Vt_module = (k_b * T_cell_kelvin / q) * module["cells"]
    IL *= pv_area / module["area_ref"]
    I0 *= pv_area / module["area_ref"]

    V = np.linspace(0, module["Voc_ref"] * 1.2, 1000)
    I = np.zeros_like(V)
    for idx, v in enumerate(V):
        I[idx] = compute_current(v, IL, I0, Rs, Rsh, a, Vt_module)
    P = I * V
    mpp_idx = np.argmax(P)
    return P[mpp_idx] if mpp_idx < len(P) else 0

def compute_current(V, IL, I0, Rs, Rsh, a, Vt_module, tolerance=1e-6, max_iterations=100):
    I = IL
    for _ in range(max_iterations):
        f = IL - I0 * (np.exp((V + I * Rs) / (a * Vt_module)) - 1) - (V + I * Rs) / Rsh - I
        df_dI = -I0 * Rs / (a * Vt_module) * np.exp((V + I * Rs) / (a * Vt_module)) - Rs / Rsh - 1
        I_new = I - f / df_dI
        if abs(I_new - I) < tolerance:
            return I_new
        I = I_new
    return I

power_output_current = np.array([compute_pv_power(gti, temp, pv_area, module) 
                               for gti, temp in zip(gti_current, cell_temperature)])
power_output_target = np.array([compute_pv_power(gti, temp, pv_area, module) 
                              for gti, temp in zip(gti_target, cell_temperature)])

# Calculate energy output
energy_output_current = power_output_current * time_interval / 1000  # kWh
energy_output_target = power_output_target * time_interval / 1000  # kWh
total_energy_current = np.sum(energy_output_current)
total_energy_target = np.sum(energy_output_target)

# Recalculate IEC Metrics
rated_power = module["Pmp_ref"]  # W
rated_power_kWp = rated_power / 1000  # kWp
total_hours = len(df_loc) * time_interval
max_possible_energy = rated_power_kWp * total_hours
theoretical_energy = np.sum(gti_current) * rated_power * time_interval / (G_STC * 1000)  # kWh

Yf_current = total_energy_current / rated_power_kWp if rated_power_kWp > 0 else 0
Yf_target = total_energy_target / rated_power_kWp if rated_power_kWp > 0 else 0
PR_current = total_energy_current / theoretical_energy if theoretical_energy > 0 else 0
PR_target = total_energy_target / theoretical_energy if theoretical_energy > 0 else 0
CF_current = total_energy_current / max_possible_energy if max_possible_energy > 0 else 0
CF_target = total_energy_target / max_possible_energy if max_possible_energy > 0 else 0

# Print recalculated IEC metrics
print("\nRecalculated IEC Metrics:")
print(f"Final Yield (Yf) - Current: {Yf_current:.2f} kWh/kWp")
print(f"Final Yield (Yf) - Target: {Yf_target:.2f} kWh/kWp")
print(f"Performance Ratio (PR) - Current: {PR_current:.4f}")
print(f"Performance Ratio (PR) - Target: {PR_target:.4f}")
print(f"Capacity Factor (CF) - Current: {CF_current:.4f}")
print(f"Capacity Factor (CF) - Target: {CF_target:.4f}")

df_test shape: (1047684, 139)
df_test columns: ['GHI(t-8)', 'DNI(t-8)', 'DHI(t-8)', 'temperature(t-8)', 'lat_sin(t-8)', 'lat_cos(t-8)', 'lon_sin(t-8)', 'lon_cos(t-8)', 'Kt(t-8)', 'Solar_Altitude(t-8)', 'Solar_Azimuth(t-8)', 'day_of_year_sin(t-8)', 'day_of_year_cos(t-8)', 'hour_sin(t-8)', 'hour_cos(t-8)', 'GHI(t-7)', 'DNI(t-7)', 'DHI(t-7)', 'temperature(t-7)', 'lat_sin(t-7)', 'lat_cos(t-7)', 'lon_sin(t-7)', 'lon_cos(t-7)', 'Kt(t-7)', 'Solar_Altitude(t-7)', 'Solar_Azimuth(t-7)', 'day_of_year_sin(t-7)', 'day_of_year_cos(t-7)', 'hour_sin(t-7)', 'hour_cos(t-7)', 'GHI(t-6)', 'DNI(t-6)', 'DHI(t-6)', 'temperature(t-6)', 'lat_sin(t-6)', 'lat_cos(t-6)', 'lon_sin(t-6)', 'lon_cos(t-6)', 'Kt(t-6)', 'Solar_Altitude(t-6)', 'Solar_Azimuth(t-6)', 'day_of_year_sin(t-6)', 'day_of_year_cos(t-6)', 'hour_sin(t-6)', 'hour_cos(t-6)', 'GHI(t-5)', 'DNI(t-5)', 'DHI(t-5)', 'temperature(t-5)', 'lat_sin(t-5)', 'lat_cos(t-5)', 'lon_sin(t-5)', 'lon_cos(t-5)', 'Kt(t-5)', 'Solar_Altitude(t-5)', 'Solar_Azimuth(t-5)', 'd

KeyError: "Zenith or Azimuth column not found. Available columns: ['GHI(t-8)', 'DNI(t-8)', 'DHI(t-8)', 'temperature(t-8)', 'lat_sin(t-8)', 'lat_cos(t-8)', 'lon_sin(t-8)', 'lon_cos(t-8)', 'Kt(t-8)', 'Solar_Altitude(t-8)', 'Solar_Azimuth(t-8)', 'day_of_year_sin(t-8)', 'day_of_year_cos(t-8)', 'hour_sin(t-8)', 'hour_cos(t-8)', 'GHI(t-7)', 'DNI(t-7)', 'DHI(t-7)', 'temperature(t-7)', 'lat_sin(t-7)', 'lat_cos(t-7)', 'lon_sin(t-7)', 'lon_cos(t-7)', 'Kt(t-7)', 'Solar_Altitude(t-7)', 'Solar_Azimuth(t-7)', 'day_of_year_sin(t-7)', 'day_of_year_cos(t-7)', 'hour_sin(t-7)', 'hour_cos(t-7)', 'GHI(t-6)', 'DNI(t-6)', 'DHI(t-6)', 'temperature(t-6)', 'lat_sin(t-6)', 'lat_cos(t-6)', 'lon_sin(t-6)', 'lon_cos(t-6)', 'Kt(t-6)', 'Solar_Altitude(t-6)', 'Solar_Azimuth(t-6)', 'day_of_year_sin(t-6)', 'day_of_year_cos(t-6)', 'hour_sin(t-6)', 'hour_cos(t-6)', 'GHI(t-5)', 'DNI(t-5)', 'DHI(t-5)', 'temperature(t-5)', 'lat_sin(t-5)', 'lat_cos(t-5)', 'lon_sin(t-5)', 'lon_cos(t-5)', 'Kt(t-5)', 'Solar_Altitude(t-5)', 'Sola